In [8]:
from math import sin, cos,asin, degrees,sqrt, atan2, radians
import json
import pandas as pd
import numpy as np
import csv
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches

## 4.2.1 Clasificación de estaciones de autobús

In [57]:
madrid_central_df = pd.read_csv('CoordenadasMadridCentral.txt', sep=",", header=0)
estaciones_dentro_df = pd.read_csv('estaciones_dentro_bicimad.csv', sep=";", header=0)

In [58]:
madrid_central_np = madrid_central_df.to_numpy()
polygon = Polygon(madrid_central_np)

In [59]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2( sqrt(a), sqrt(1-a) )
    return R * c * 1000

In [60]:
def is_in_madrid_central(lat, lon,ratio):
    point = Point(lon,lat)
    if is_in_limit_madrid_central(lat,lon,ratio): return 2
    else: 
        if polygon.contains(point): return 1
        else: return 0

In [61]:
def is_in_limit_madrid_central(lat,lon,radio):
    for index, row in madrid_central_df.iterrows():
        distancia = calcular_distancia(lat,lon,row.latitude,row.longitude) 
        if(distancia <= radio): return True
    return False

In [62]:
def discriminate_madrid_central(df):
    aux = []
    for index, row in df.iterrows():
        row_aux = row.tolist()
        row_aux.append(is_in_madrid_central(row.latitude,row.longitude,25))
        aux.append(row_aux)
    return aux

In [63]:
pd.DataFrame(discriminate_madrid_central(estaciones_dentro_df), columns = ["id_line","label","type_header","id_header","name_header","latitude","longitude","mc"]).to_csv('mc_estaciones_dentro.csv', sep=';', encoding='utf-8',index=False)

Muestra de las primeras cinco estaciones

In [74]:
estaciones_emt_mc = pd.read_csv('mc_estaciones_dentro.csv', sep=";", header=0)

In [75]:
estaciones_emt_mc.head(5)

,id_line,label,type_header,id_header,name_header,latitude,longitude,mc
0,5,5,headerA,5907,Sol-Sevilla,40.418008,-3.698486,1
1,15,15,headerA,2711,Sol-Sevilla,40.417495,-3.698388,1
2,20,20,headerA,2711,Sol-Sevilla,40.417495,-3.698388,1
3,46,46,headerA,2056,Sevilla,40.418253,-3.698951,1
4,51,51,headerA,5682,Sol,40.417264,-3.702258,1


## 4.2.2 Clasificación de estaciones de bicicleta

In [68]:
bicimad_df = pd.read_csv('bicimad_stations.csv', sep=";", header=0)

In [71]:
pd.DataFrame(discriminate_madrid_central(bicimad_df), columns = ["id","number","name","address","activate","no_available","total_bases","longitude","latitude","mc"]).to_csv('mc_bicimad.csv', sep=';', encoding='utf-8',index=False)

Muestra de las primeras cinco estaciones

In [76]:
bicimad_dentro_mc = pd.read_csv('mc_bicimad.csv', sep=";", header=0)

In [77]:
estaciones_bicimad_totales.head(5)

,id_line,label,type_header,id_header,name_header,latitude,longitude,mc
0,5,5,headerA,5907,Sol-Sevilla,40.418008,-3.698486,1
1,15,15,headerA,2711,Sol-Sevilla,40.417495,-3.698388,1
2,20,20,headerA,2711,Sol-Sevilla,40.417495,-3.698388,1
3,46,46,headerA,2056,Sevilla,40.418253,-3.698951,1
4,51,51,headerA,5682,Sol,40.417264,-3.702258,1


## 4.2.3 Clasificación de las líneas que empiecen y terminen dentro de la zona

In [78]:
mc_in = estaciones_emt_mc[estaciones_emt_mc.mc != 0].sort_values('id_line')
mc_out = estaciones_emt_mc[estaciones_emt_mc.mc == 0].sort_values('id_line')

In [79]:
def get_lines(df):
    iguales = df.groupby('id_line')
    res = iguales.filter(lambda x: len(x) >= 2)
    return res

In [80]:
def get_mc_cerrado(df):
    return get_lines(df)

In [81]:
mc_cabeceras_cerrado = get_mc_cerrado(mc_in)

##### Número de lineas encontradas

In [82]:
mc_cabeceras_cerrado

,id_line,label,type_header,id_header,name_header,latitude,longitude,mc
58,362,OO2,headerA,80,Metro Puerta de Toledo,40.407200,-3.711269,1
74,362,OO2,headerB,289,Arguelles,40.430514,-3.715236,2
66,601,M1,headerB,4057,Embajadores,40.405569,-3.702604,1
7,601,M1,headerA,5837,Sol-Sevilla,40.417408,-3.700132,1


## 4.2.4 Clasificación de las líneas que empiecen y terminen cerca de la zona

In [91]:
def loop_distancia(latitud,longitud,radio,df):
    res = []
    for index, row in df.iterrows():
        distancia = calcular_distancia(latitud,longitud,row.latitude,row.longitude) 
        if distancia <= radio:
            res.append(row)
    return pd.DataFrame(res) 

In [87]:
def calcular_distancia_con_coordinates(lat,lon,radio,df):
    res = loop_distancia(lat,lon,radio,df)
    return res

In [88]:
def incluir_estaciones_cercanas_mc(radio,cabeceras_df):
    estaciones_dentro_bicimad = []
    for index, row in madrid_central_df.iterrows():
        res = calcular_distancia_con_coordinates(row.latitude,row.longitude,radio,cabeceras_df)
        for index, r in res.iterrows():
            r = r.tolist()
            if r not in estaciones_dentro_bicimad:
                estaciones_dentro_bicimad.append(r)
    return pd.DataFrame(estaciones_dentro_bicimad,columns=["id_line","label","type_header","id_header","name_header","latitude","longitude","mc"])

In [89]:
def get_mc_abierto(radio,df):
    res = incluir_estaciones_cercanas_mc(radio,df)
    return get_lines(pd.concat([mc_in,res]).sort_values('id_line'))

In [92]:
mc_cabeceras_abierto = get_mc_abierto(300,mc_out)

##### Número de lineas encontradas

In [93]:
mc_cabeceras_abierto

,id_line,label,type_header,id_header,name_header,latitude,longitude,mc
74,362,OO2,headerB,289,Arguelles,40.430514,-3.715236,2
58,362,OO2,headerA,80,Metro Puerta de Toledo,40.407200,-3.711269,1
7,601,M1,headerA,5837,Sol-Sevilla,40.417408,-3.700132,1
66,601,M1,headerB,4057,Embajadores,40.405569,-3.702604,1
